In [82]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, TimeDistributed, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split






In [77]:
#  recurso necessários do NLTK
nltk.download('punkt')

def read_conll_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    frases = []
    tags = []
    frase_atual = []
    tags_atual = []

    for line in lines:
        if line.strip() == '':
            if frase_atual and tags_atual:
                frases.append(frase_atual)
                tags.append(tags_atual)
                frase_atual = []
                tags_atual = []
        else:
            palavra, _, _, tag = line.strip().split()
            frase_atual.append(palavra)
            tags_atual.append(tag)

    return frases, tags

# dados de treinamento e teste
frases_train, tags_train = read_conll_data('/content/train.txt')
frases_test, tags_test = read_conll_data('/content/test.txt')

#  vocabulário de palavras e tags
vocabulario_palavras = set(word for frase in frases_train for word in frase)
vocabulario_tags = set(tag for tags_frase in tags_train for tag in tags_frase)

#  dicionários para mapear palavras e tags para índices
palavra_para_indice = {palavra: i + 1 for i, palavra in enumerate(vocabulario_palavras)}
tag_para_indice = {tag: i for i, tag in enumerate(vocabulario_tags)}

# Função para converter frases e tags em sequências de índices
def process_data(frases, tags, palavra_para_indice, tag_para_indice, max_len):
    frases_indices = [[palavra_para_indice.get(palavra, 0) for palavra in frase] for frase in frases]
    tags_indices = [[tag_para_indice[tag] for tag in tags_frase] for tags_frase in tags]

    frases_preenchidas = pad_sequences(frases_indices, maxlen=max_len, padding='post')
    tags_preenchidas = pad_sequences(tags_indices, maxlen=max_len, padding='post')

    tags_categoricas = [to_categorical(tags_frase, num_classes=len(vocabulario_tags)) for tags_frase in tags_preenchidas]

    return frases_preenchidas, np.array(tags_categoricas)

# Encontra o comprimento máximo das sequências em ambos os conjuntos de dados
max_len = max(max(len(frase) for frase in frases_train), max(len(frase) for frase in frases_test))

# Pré-processamento os dados de treinamento e teste
X_train, y_train = process_data(frases_train, tags_train, palavra_para_indice, tag_para_indice, max_len)
X_test, y_test = process_data(frases_test, tags_test, palavra_para_indice, tag_para_indice, max_len)

#  modelo de NER
modelo = Sequential()
modelo.add(Embedding(input_dim=len(vocabulario_palavras) + 1, output_dim=32, input_length=max_len))
modelo.add(Bidirectional(LSTM(units=32, return_sequences=True)))
modelo.add(TimeDistributed(Dense(len(vocabulario_tags), activation='softmax')))

modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


modelo.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1/10
469/469 [==============================] - 80s 162ms/step - loss: 0.2117 - accuracy: 0.9625 - val_loss: 0.0995 - val_accuracy: 0.9738
Epoch 2/10
469/469 [==============================] - 66s 140ms/step - loss: 0.0544 - accuracy: 0.9834 - val_loss: 0.0797 - val_accuracy: 0.9772
Epoch 3/10
469/469 [==============================] - 65s 138ms/step - loss: 0.0330 - accuracy: 0.9903 - val_loss: 0.0820 - val_accuracy: 0.9813
Epoch 4/10
469/469 [==============================] - 62s 131ms/step - loss: 0.0183 - accuracy: 0.9955 - val_loss: 0.0866 - val_accuracy: 0.9838
Epoch 5/10
469/469 [==============================] - 63s 135ms/step - loss: 0.0088 - accuracy: 0.9982 - val_loss: 0.0945 - val_accuracy: 0.9843
Epoch 6/10
469/469 [==============================] - 63s 135ms/step - loss: 0.0049 - accuracy: 0.9991 - val_loss: 0.1039 - val_accuracy: 0.9837
Epoch 7/10
469/469 [==============================] - 64s 137ms/step - loss: 0.0031 - accuracy: 0.9994 - val_loss: 0.1103 - val_ac

In [80]:
def extrair_entidades(texto, modelo, palavra_para_indice, indice_para_tag, max_len):

    palavras = word_tokenize(texto)

    # Converte as palavras em índices
    palavra_indices = [palavra_para_indice.get(palavra, 0) for palavra in palavras]

    # Preenche a sequência de índices
    palavra_indices_preenchidas = pad_sequences([palavra_indices], maxlen=max_len, padding='post')

    # Usao modelo para prever as tags
    tags_predicao = modelo.predict(palavra_indices_preenchidas)

    # Converte as tags previstas em strings
    tags_predicao_strings = [indice_para_tag[np.argmax(tag)] for tag in tags_predicao[0][:len(palavras)]]

    # Extrai as entidades nomeadas
    entidades = []
    entidade_atual = []
    tag_atual = None

    for palavra, tag in zip(palavras, tags_predicao_strings):
        if tag.startswith('B-'):
            if entidade_atual:
                entidades.append((' '.join(entidade_atual), tag_atual))
                entidade_atual = []
            tag_atual = tag[2:]
            entidade_atual.append(palavra)
        elif tag.startswith('I-') and tag[2:] == tag_atual:
            entidade_atual.append(palavra)
        else:
            if entidade_atual:
                entidades.append((' '.join(entidade_atual), tag_atual))
                entidade_atual = []
                tag_atual = None

    if entidade_atual:
        entidades.append((' '.join(entidade_atual), tag_atual))

    return entidades

# Cria um dicionário para mapear índices para tags
indice_para_tag = {i: tag for tag, i in tag_para_indice.items()}


texto = """Last week, Alice Johnson, a software engineer from San Francisco, attended the annual Tech Innovators Conference held at the Moscone Center.
 During the event, she met John Smith, the CEO of FutureTech Inc., a leading company in artificial intelligence and robotics.
  They discussed the latest advancements in machine learning and the potential applications of AI in various industries, such as healthcare and finance.
   Alice was particularly interested in the presentation given by Dr. Jane Doe from Stanford University, who shared her research on natural language processing and its implications for the future of human-computer interaction."""

palavras = word_tokenize(texto)



entidades = extrair_entidades(texto, modelo, palavra_para_indice, indice_para_tag, max_len)

print("Entidades extraídas:", entidades)

1/1 [==============================] - 0s 49ms/step
Entidades extraídas: [('Alice Johnson', 'PER'), ('San Francisco', 'LOC'), ('Tech Innovators Conference held', 'ORG'), ('John Smith', 'PER'), ('robotics', 'ORG'), ('AI', 'ORG'), ('presentation given', 'ORG'), ('Jane', 'PER'), ('implications for the', 'ORG')]
